In [2]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.1-bin-hadoop3.2"

!ls

$RECYCLE.BIN
Cover Letter and Classes Schedule.pdf
Cover Letter.pdf
Exam1_AMohd.docx
RafayIBDDS.ipynb
Student-Employment-Application-Feb-2021.pdf
Texas DPS - Schedule Appointment.pdf
Thumbs.db
View My Classes.pdf
Ziya.ipynb
desktop.ini
weatherAUS.csv
~$actice Writing.docx
~$hd abdul rafay.docx


<span style="font-size: 36px;color: teal; font-family: TimesNewRoman;">Creating a Spark Session<span>

In [42]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Assignment").getOrCreate()
# spark.conf.set("spark.sql.repl.eagerEval.enabled", True) # Property used to format output tables better
spark

<span style="font-size: 36px;color: teal; font-family: TimesNewRoman;">Loading a Dataset<span>

In [43]:
# Load data from csv to a dataframe.
# header=True means the first row is a header
# sep=';' means the column are seperated using ''
df_spark = spark.read.csv('weatherAUS.csv', header=True, sep=";")
df_spark.show()

+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|Date,Location,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustDir,WindGustSpeed,WindDir9am,WindDir3pm,WindSpeed9am,WindSpeed3pm,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm,RainToday,RISK_MM,RainTomorrow|
+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|                                                                                                                                                                                                                                  2008-12-01,Albury...|
|   

In [44]:
df_pyspark = spark.read.csv('weatherAUS.csv', header = True,  inferSchema = True)
df_pyspark.show()

+----------+--------+-------+-------+--------+-----------+--------+-----------+-------------+----------+----------+------------+------------+-----------+-----------+-----------+-----------+--------+--------+-------+-------+---------+-------+------------+
|      Date|Location|MinTemp|MaxTemp|Rainfall|Evaporation|Sunshine|WindGustDir|WindGustSpeed|WindDir9am|WindDir3pm|WindSpeed9am|WindSpeed3pm|Humidity9am|Humidity3pm|Pressure9am|Pressure3pm|Cloud9am|Cloud3pm|Temp9am|Temp3pm|RainToday|RISK_MM|RainTomorrow|
+----------+--------+-------+-------+--------+-----------+--------+-----------+-------------+----------+----------+------------+------------+-----------+-----------+-----------+-----------+--------+--------+-------+-------+---------+-------+------------+
|2008-12-01|  Albury|   13.4|   22.9|     0.6|         NA|      NA|          W|           44|         W|       WNW|          20|          24|         71|         22|     1007.7|     1007.1|       8|      NA|   16.9|   21.8|       No|  

In [45]:
type(df_pyspark)

pyspark.sql.dataframe.DataFrame

In [46]:
df_pyspark.printSchema()

root
 |-- Date: date (nullable = true)
 |-- Location: string (nullable = true)
 |-- MinTemp: string (nullable = true)
 |-- MaxTemp: string (nullable = true)
 |-- Rainfall: string (nullable = true)
 |-- Evaporation: string (nullable = true)
 |-- Sunshine: string (nullable = true)
 |-- WindGustDir: string (nullable = true)
 |-- WindGustSpeed: string (nullable = true)
 |-- WindDir9am: string (nullable = true)
 |-- WindDir3pm: string (nullable = true)
 |-- WindSpeed9am: string (nullable = true)
 |-- WindSpeed3pm: string (nullable = true)
 |-- Humidity9am: string (nullable = true)
 |-- Humidity3pm: string (nullable = true)
 |-- Pressure9am: string (nullable = true)
 |-- Pressure3pm: string (nullable = true)
 |-- Cloud9am: string (nullable = true)
 |-- Cloud3pm: string (nullable = true)
 |-- Temp9am: string (nullable = true)
 |-- Temp3pm: string (nullable = true)
 |-- RainToday: string (nullable = true)
 |-- RISK_MM: double (nullable = true)
 |-- RainTomorrow: string (nullable = true)



In [47]:
df_pyspark.head()

Row(Date=datetime.date(2008, 12, 1), Location='Albury', MinTemp='13.4', MaxTemp='22.9', Rainfall='0.6', Evaporation='NA', Sunshine='NA', WindGustDir='W', WindGustSpeed='44', WindDir9am='W', WindDir3pm='WNW', WindSpeed9am='20', WindSpeed3pm='24', Humidity9am='71', Humidity3pm='22', Pressure9am='1007.7', Pressure3pm='1007.1', Cloud9am='8', Cloud3pm='NA', Temp9am='16.9', Temp3pm='21.8', RainToday='No', RISK_MM=0.0, RainTomorrow='No')

In [48]:
df_pyspark.describe().show()

+-------+--------+------------------+------------------+------------------+-----------------+------------------+-----------+------------------+----------+----------+-----------------+-----------------+----------------+------------------+-----------------+------------------+-----------------+------------------+-----------------+-----------------+---------+-----------------+------------+
|summary|Location|           MinTemp|           MaxTemp|          Rainfall|      Evaporation|          Sunshine|WindGustDir|     WindGustSpeed|WindDir9am|WindDir3pm|     WindSpeed9am|     WindSpeed3pm|     Humidity9am|       Humidity3pm|      Pressure9am|       Pressure3pm|         Cloud9am|          Cloud3pm|          Temp9am|          Temp3pm|RainToday|          RISK_MM|RainTomorrow|
+-------+--------+------------------+------------------+------------------+-----------------+------------------+-----------+------------------+----------+----------+-----------------+-----------------+----------------+----

<span style="font-size: 36px;color: teal; font-family: TimesNewRoman;">Question 1:	Explore the dataset to find what are the categorical variables in the dataset. List categorical variable names<span>

In [49]:
# List categorical variable names (string columns)
categorical_columns = [col for col, dtype in df_pyspark.dtypes if dtype == "string"]

print("Categorical variable names:")
print(categorical_columns)

Categorical variable names:
['Location', 'MinTemp', 'MaxTemp', 'Rainfall', 'Evaporation', 'Sunshine', 'WindGustDir', 'WindGustSpeed', 'WindDir9am', 'WindDir3pm', 'WindSpeed9am', 'WindSpeed3pm', 'Humidity9am', 'Humidity3pm', 'Pressure9am', 'Pressure3pm', 'Cloud9am', 'Cloud3pm', 'Temp9am', 'Temp3pm', 'RainToday', 'RainTomorrow']


<span style="font-size: 36px;color: teal; font-family: TimesNewRoman;">Question 2:	Explore how many categorical variables have null values. <span>

In [50]:
from pyspark.sql.functions import col
# Define the list of string columns
string_columns = [col_name for col_name, data_type in df_pyspark.dtypes if data_type == "string"]

# Iterate over each string column and count rows with NA values
na_counts = {}
for col_name in string_columns:
    na_count = df_pyspark.filter((col(col_name).isNull()) | (col(col_name) == "NA")).count()
    na_counts[col_name] = na_count

# Print the counts of rows with NA values for each string column
for col_name, count in na_counts.items():
    print(f"Column '{col_name}' has {count} rows with NA values.")

Column 'Location' has 0 rows with NA values.
Column 'MinTemp' has 637 rows with NA values.
Column 'MaxTemp' has 322 rows with NA values.
Column 'Rainfall' has 1406 rows with NA values.
Column 'Evaporation' has 60843 rows with NA values.
Column 'Sunshine' has 67816 rows with NA values.
Column 'WindGustDir' has 9330 rows with NA values.
Column 'WindGustSpeed' has 9270 rows with NA values.
Column 'WindDir9am' has 10013 rows with NA values.
Column 'WindDir3pm' has 3778 rows with NA values.
Column 'WindSpeed9am' has 1348 rows with NA values.
Column 'WindSpeed3pm' has 2630 rows with NA values.
Column 'Humidity9am' has 1774 rows with NA values.
Column 'Humidity3pm' has 3610 rows with NA values.
Column 'Pressure9am' has 14014 rows with NA values.
Column 'Pressure3pm' has 13981 rows with NA values.
Column 'Cloud9am' has 53657 rows with NA values.
Column 'Cloud3pm' has 57094 rows with NA values.
Column 'Temp9am' has 904 rows with NA values.
Column 'Temp3pm' has 2726 rows with NA values.
Column '

<span style="font-size: 36px;color: teal; font-family: TimesNewRoman;">Question 3: Get the frequency count of each categorical variable. For instance, how many discrete values in each categorical variable and how many datapoints from each distinct value.<span>

In [51]:

# Iterate over each column in the DataFrame
for col in df_pyspark.columns:
    # Calculate the frequency count of categories in the column
    freq_count = df_pyspark.groupBy(col).count()
    
    # Print the result
    print(f"Frequency count of categories in column '{col}':")
    freq_count.show(truncate=False)


Frequency count of categories in column 'Date':
+----------+-----+
|Date      |count|
+----------+-----+
|2009-07-25|45   |
|2009-11-22|46   |
|2010-08-11|45   |
|2012-04-17|46   |
|2012-10-06|41   |
|2013-01-22|46   |
|2013-03-26|49   |
|2013-05-21|47   |
|2013-09-09|48   |
|2014-09-26|47   |
|2014-11-12|48   |
|2015-03-09|48   |
|2015-05-19|45   |
|2016-03-01|48   |
|2009-06-28|46   |
|2009-08-09|46   |
|2009-09-27|45   |
|2009-10-19|46   |
|2010-09-24|46   |
|2011-01-29|44   |
+----------+-----+
only showing top 20 rows

Frequency count of categories in column 'Location':
+-------------+-----+
|Location     |count|
+-------------+-----+
|NorfolkIsland|2964 |
|Cobar        |2988 |
|SydneyAirport|3005 |
|Wollongong   |2983 |
|Williamtown  |2553 |
|Moree        |2854 |
|Sydney       |3337 |
|BadgerysCreek|2928 |
|Albury       |3011 |
|WaggaWagga   |2976 |
|CoffsHarbour |2953 |
|Penrith      |2964 |
|NorahHead    |2929 |
|Newcastle    |2955 |
|Richmond     |2951 |
|Bendigo      |3034 |


<span style="font-size: 36px;color: teal; font-family: TimesNewRoman;">Question 4: Print the first five rows of the dataset.<span>

In [52]:
df_pyspark.show(5)

+----------+--------+-------+-------+--------+-----------+--------+-----------+-------------+----------+----------+------------+------------+-----------+-----------+-----------+-----------+--------+--------+-------+-------+---------+-------+------------+
|      Date|Location|MinTemp|MaxTemp|Rainfall|Evaporation|Sunshine|WindGustDir|WindGustSpeed|WindDir9am|WindDir3pm|WindSpeed9am|WindSpeed3pm|Humidity9am|Humidity3pm|Pressure9am|Pressure3pm|Cloud9am|Cloud3pm|Temp9am|Temp3pm|RainToday|RISK_MM|RainTomorrow|
+----------+--------+-------+-------+--------+-----------+--------+-----------+-------------+----------+----------+------------+------------+-----------+-----------+-----------+-----------+--------+--------+-------+-------+---------+-------+------------+
|2008-12-01|  Albury|   13.4|   22.9|     0.6|         NA|      NA|          W|           44|         W|       WNW|          20|          24|         71|         22|     1007.7|     1007.1|       8|      NA|   16.9|   21.8|       No|  

<span style="font-size: 36px;color: teal; font-family: TimesNewRoman;">Question 5: What are the available columns of the dataset?<span>

In [53]:
df_pyspark.columns

['Date',
 'Location',
 'MinTemp',
 'MaxTemp',
 'Rainfall',
 'Evaporation',
 'Sunshine',
 'WindGustDir',
 'WindGustSpeed',
 'WindDir9am',
 'WindDir3pm',
 'WindSpeed9am',
 'WindSpeed3pm',
 'Humidity9am',
 'Humidity3pm',
 'Pressure9am',
 'Pressure3pm',
 'Cloud9am',
 'Cloud3pm',
 'Temp9am',
 'Temp3pm',
 'RainToday',
 'RISK_MM',
 'RainTomorrow']

<span style="font-size: 36px;color: teal; font-family: TimesNewRoman;">Question 6:	Drop RISK_MM variable column.<span>

In [54]:
new_df = df_pyspark.drop('RISK_MM')
new_df.columns

['Date',
 'Location',
 'MinTemp',
 'MaxTemp',
 'Rainfall',
 'Evaporation',
 'Sunshine',
 'WindGustDir',
 'WindGustSpeed',
 'WindDir9am',
 'WindDir3pm',
 'WindSpeed9am',
 'WindSpeed3pm',
 'Humidity9am',
 'Humidity3pm',
 'Pressure9am',
 'Pressure3pm',
 'Cloud9am',
 'Cloud3pm',
 'Temp9am',
 'Temp3pm',
 'RainToday',
 'RainTomorrow']

<span style="font-size: 36px;color: teal; font-family: TimesNewRoman;">Question 7:	Get the summary of the dataset.<span>

In [55]:
new_df.describe().show()

+-------+--------+------------------+------------------+------------------+-----------------+------------------+-----------+------------------+----------+----------+-----------------+-----------------+----------------+------------------+-----------------+------------------+-----------------+------------------+-----------------+-----------------+---------+------------+
|summary|Location|           MinTemp|           MaxTemp|          Rainfall|      Evaporation|          Sunshine|WindGustDir|     WindGustSpeed|WindDir9am|WindDir3pm|     WindSpeed9am|     WindSpeed3pm|     Humidity9am|       Humidity3pm|      Pressure9am|       Pressure3pm|         Cloud9am|          Cloud3pm|          Temp9am|          Temp3pm|RainToday|RainTomorrow|
+-------+--------+------------------+------------------+------------------+-----------------+------------------+-----------+------------------+----------+----------+-----------------+-----------------+----------------+------------------+-----------------+---

<span style="font-size: 36px;color: teal; font-family: TimesNewRoman;">Question 8: List first five rows only for categorical variables.<span>

In [166]:
df_drop = df_pyspark.drop('Date','RISK_MM')
df_drop.show(5)

+--------+-------+-------+--------+-----------+--------+-----------+-------------+----------+----------+------------+------------+-----------+-----------+-----------+-----------+--------+--------+-------+-------+---------+------------+
|Location|MinTemp|MaxTemp|Rainfall|Evaporation|Sunshine|WindGustDir|WindGustSpeed|WindDir9am|WindDir3pm|WindSpeed9am|WindSpeed3pm|Humidity9am|Humidity3pm|Pressure9am|Pressure3pm|Cloud9am|Cloud3pm|Temp9am|Temp3pm|RainToday|RainTomorrow|
+--------+-------+-------+--------+-----------+--------+-----------+-------------+----------+----------+------------+------------+-----------+-----------+-----------+-----------+--------+--------+-------+-------+---------+------------+
|  Albury|   13.4|   22.9|     0.6|         NA|      NA|          W|           44|         W|       WNW|          20|          24|         71|         22|     1007.7|     1007.1|       8|      NA|   16.9|   21.8|       No|          No|
|  Albury|    7.4|   25.1|       0|         NA|      NA|

<span style="font-size: 36px;color: teal; font-family: TimesNewRoman;">Question 9: Decompose the date field into year, month, and day fields. Then, drop the original date field.<span>

In [56]:
from pyspark.sql.functions import year, month, dayofmonth

# Decompose the date column into year, month, and day fields
df = new_df.withColumn("year", year("Date")) \
       .withColumn("month", month("Date")) \
       .withColumn("day", dayofmonth("Date"))

# Drop the original date column
df =  df.drop("Date")
df.show()

+--------+-------+-------+--------+-----------+--------+-----------+-------------+----------+----------+------------+------------+-----------+-----------+-----------+-----------+--------+--------+-------+-------+---------+------------+----+-----+---+
|Location|MinTemp|MaxTemp|Rainfall|Evaporation|Sunshine|WindGustDir|WindGustSpeed|WindDir9am|WindDir3pm|WindSpeed9am|WindSpeed3pm|Humidity9am|Humidity3pm|Pressure9am|Pressure3pm|Cloud9am|Cloud3pm|Temp9am|Temp3pm|RainToday|RainTomorrow|year|month|day|
+--------+-------+-------+--------+-----------+--------+-----------+-------------+----------+----------+------------+------------+-----------+-----------+-----------+-----------+--------+--------+-------+-------+---------+------------+----+-----+---+
|  Albury|   13.4|   22.9|     0.6|         NA|      NA|          W|           44|         W|       WNW|          20|          24|         71|         22|     1007.7|     1007.1|       8|      NA|   16.9|   21.8|       No|          No|2008|   12| 

<span style="font-size: 36px;color: teal; font-family: TimesNewRoman;">Question 10: How many unique locations are there in the dataset?<span>

In [57]:
unique = df.select('Location').distinct().count()
unique

49

<span style="font-size: 36px;color: teal; font-family: TimesNewRoman;">Question 11:	Print the number of times each unique location appears in the dataset.<span>

In [58]:
location_frequency = df.groupby('Location').count()
location_frequency.show()

+-------------+-----+
|     Location|count|
+-------------+-----+
|NorfolkIsland| 2964|
|        Cobar| 2988|
|SydneyAirport| 3005|
|   Wollongong| 2983|
|  Williamtown| 2553|
|        Moree| 2854|
|       Sydney| 3337|
|BadgerysCreek| 2928|
|       Albury| 3011|
|   WaggaWagga| 2976|
| CoffsHarbour| 2953|
|      Penrith| 2964|
|    NorahHead| 2929|
|    Newcastle| 2955|
|     Richmond| 2951|
|      Bendigo| 3034|
|     Canberra| 3418|
|      Mildura| 3007|
|     Portland| 2996|
|     Brisbane| 3161|
+-------------+-----+
only showing top 20 rows



In [59]:
df.printSchema()

root
 |-- Location: string (nullable = true)
 |-- MinTemp: string (nullable = true)
 |-- MaxTemp: string (nullable = true)
 |-- Rainfall: string (nullable = true)
 |-- Evaporation: string (nullable = true)
 |-- Sunshine: string (nullable = true)
 |-- WindGustDir: string (nullable = true)
 |-- WindGustSpeed: string (nullable = true)
 |-- WindDir9am: string (nullable = true)
 |-- WindDir3pm: string (nullable = true)
 |-- WindSpeed9am: string (nullable = true)
 |-- WindSpeed3pm: string (nullable = true)
 |-- Humidity9am: string (nullable = true)
 |-- Humidity3pm: string (nullable = true)
 |-- Pressure9am: string (nullable = true)
 |-- Pressure3pm: string (nullable = true)
 |-- Cloud9am: string (nullable = true)
 |-- Cloud3pm: string (nullable = true)
 |-- Temp9am: string (nullable = true)
 |-- Temp3pm: string (nullable = true)
 |-- RainToday: string (nullable = true)
 |-- RainTomorrow: string (nullable = true)
 |-- year: integer (nullable = true)
 |-- month: integer (nullable = true)
 |--

<span style="font-size: 36px;color: teal; font-family: TimesNewRoman;">Converting the datatypes of numerical columns to their respective datatypes<span>

In [95]:
from pyspark.sql.functions import col
string_columns = ['MinTemp','MaxTemp','Rainfall','Evaporation','Sunshine','WindGustSpeed','WindSpeed9am','WindSpeed3pm','Humidity9am','Humidity3pm',
                  'Pressure9am','Pressure3pm','Cloud9am','Cloud3pm','Temp9am','Temp3pm','year','month','day']
for col_name in string_columns:
    df = df.withColumn(col_name, col(col_name).cast('double'))

df.printSchema()


root
 |-- Location: string (nullable = true)
 |-- MinTemp: double (nullable = true)
 |-- MaxTemp: double (nullable = true)
 |-- Rainfall: double (nullable = true)
 |-- Evaporation: double (nullable = true)
 |-- Sunshine: double (nullable = true)
 |-- WindGustDir: string (nullable = true)
 |-- WindGustSpeed: double (nullable = true)
 |-- WindDir9am: string (nullable = true)
 |-- WindDir3pm: string (nullable = true)
 |-- WindSpeed9am: double (nullable = true)
 |-- WindSpeed3pm: double (nullable = true)
 |-- Humidity9am: double (nullable = true)
 |-- Humidity3pm: double (nullable = true)
 |-- Pressure9am: double (nullable = true)
 |-- Pressure3pm: double (nullable = true)
 |-- Cloud9am: double (nullable = true)
 |-- Cloud3pm: double (nullable = true)
 |-- Temp9am: double (nullable = true)
 |-- Temp3pm: double (nullable = true)
 |-- RainToday: string (nullable = true)
 |-- RainTomorrow: string (nullable = true)
 |-- year: double (nullable = true)
 |-- month: double (nullable = true)
 |-- d

<span style="font-size: 36px;color: teal; font-family: TimesNewRoman;"><li style="font-size: 36px;color: teal; font-family: TimesNewRoman;">Question 14:	You may use the median value (median imputation) of each field to fill null values if it is a numerical column. Use the most frequent value in case of a categorical column<li style="font-size: 36px;color: teal; font-family: TimesNewRoman;">Ans: Finding the median and replacing null values for all the numerical columns using the Imputer function from Pyspark<span>

In [109]:
# Handling missing values for numerical columns using the median of the values in the particular columns
from pyspark.ml.feature import Imputer
median_df = df
for col in string_columns:
    imputer = Imputer(inputCol=col,outputCol=col+'_median ',strategy="median")
    median_df = imputer.fit(median_df).transform(median_df)

median_df = median_df.drop(*string_columns)
median_df.show()

+--------+-----------+----------+----------+---------+------------+---------------+---------------+----------------+-------------------+----------------+---------------------+--------------------+--------------------+-------------------+-------------------+-------------------+-------------------+----------------+----------------+---------------+---------------+------------+-------------+-----------+
|Location|WindGustDir|WindDir9am|WindDir3pm|RainToday|RainTomorrow|MinTemp_median |MaxTemp_median |Rainfall_median |Evaporation_median |Sunshine_median |WindGustSpeed_median |WindSpeed9am_median |WindSpeed3pm_median |Humidity9am_median |Humidity3pm_median |Pressure9am_median |Pressure3pm_median |Cloud9am_median |Cloud3pm_median |Temp9am_median |Temp3pm_median |year_median |month_median |day_median |
+--------+-----------+----------+----------+---------+------------+---------------+---------------+----------------+-------------------+----------------+---------------------+-------------------

<span style="font-size: 36px;color: teal; font-family: TimesNewRoman;">Ans: Converting the categorical columns to indexed columns using StringIndexer from Pyspark<span>

In [110]:
from pyspark.ml.feature import StringIndexer, OneHotEncoder, VectorAssembler
inputColumns = ['Location','WindGustDir','WindDir9am','WindDir3pm','RainToday','RainTomorrow']
indexed_df = median_df
for col in inputColumns:
    string_indexer = StringIndexer(inputCol=col,outputCol=col+'_indexed',handleInvalid="keep")
    indexed_df = string_indexer.fit(indexed_df).transform(indexed_df)

indexed_df = indexed_df.drop(*inputColumns)
indexed_df.show()

+---------------+---------------+----------------+-------------------+----------------+---------------------+--------------------+--------------------+-------------------+-------------------+-------------------+-------------------+----------------+----------------+---------------+---------------+------------+-------------+-----------+----------------+-------------------+------------------+------------------+-----------------+--------------------+
|MinTemp_median |MaxTemp_median |Rainfall_median |Evaporation_median |Sunshine_median |WindGustSpeed_median |WindSpeed9am_median |WindSpeed3pm_median |Humidity9am_median |Humidity3pm_median |Pressure9am_median |Pressure3pm_median |Cloud9am_median |Cloud3pm_median |Temp9am_median |Temp3pm_median |year_median |month_median |day_median |Location_indexed|WindGustDir_indexed|WindDir9am_indexed|WindDir3pm_indexed|RainToday_indexed|RainTomorrow_indexed|
+---------------+---------------+----------------+-------------------+----------------+-----------

<span style="font-size: 36px;color: teal; font-family: TimesNewRoman;">Ans: Now finding the mode for the converted categorical columns to indexed columns to replace it with null values<span>

In [111]:
# Handling the missing values for the categorical columns using the mode imputation after the string indexing
mode_df = indexed_df
inputColumns = ['Location_indexed','WindGustDir_indexed','WindDir9am_indexed','WindDir3pm_indexed','RainToday_indexed','RainTomorrow_indexed']
for col in inputColumns:
    imputer = Imputer(inputCol=col,outputCol=col+'_mode',strategy="mode")
    mode_df = imputer.fit(mode_df).transform(mode_df)

mode_df = mode_df.drop(*inputColumns)
mode_df.show()

+---------------+---------------+----------------+-------------------+----------------+---------------------+--------------------+--------------------+-------------------+-------------------+-------------------+-------------------+----------------+----------------+---------------+---------------+------------+-------------+-----------+---------------------+------------------------+-----------------------+-----------------------+----------------------+-------------------------+
|MinTemp_median |MaxTemp_median |Rainfall_median |Evaporation_median |Sunshine_median |WindGustSpeed_median |WindSpeed9am_median |WindSpeed3pm_median |Humidity9am_median |Humidity3pm_median |Pressure9am_median |Pressure3pm_median |Cloud9am_median |Cloud3pm_median |Temp9am_median |Temp3pm_median |year_median |month_median |day_median |Location_indexed_mode|WindGustDir_indexed_mode|WindDir9am_indexed_mode|WindDir3pm_indexed_mode|RainToday_indexed_mode|RainTomorrow_indexed_mode|
+---------------+---------------+-----

<span style="font-size: 36px;color: teal; font-family: TimesNewRoman;">Question 12:	Perform One Hot Encoding for each categorical variable.<span>

In [134]:
indexed_mode_columns = ['Location_indexed_mode','WindGustDir_indexed_mode','WindDir9am_indexed_mode','WindDir3pm_indexed_mode',
                   'RainToday_indexed_mode']
encoded_df = mode_df
for col in indexed_mode_columns:
    encoded_columns = OneHotEncoder(inputCol=col,outputCol=col+'_encoded')
    encoded_df = encoded_columns.fit(encoded_df).transform(encoded_df)

encoded_df = encoded_df.drop(*indexed_mode_columns)
encoded_df.show()


+---------------+---------------+----------------+-------------------+----------------+---------------------+--------------------+--------------------+-------------------+-------------------+-------------------+-------------------+----------------+----------------+---------------+---------------+------------+-------------+-----------+-------------------------+-----------------------------+--------------------------------+-------------------------------+-------------------------------+------------------------------+
|MinTemp_median |MaxTemp_median |Rainfall_median |Evaporation_median |Sunshine_median |WindGustSpeed_median |WindSpeed9am_median |WindSpeed3pm_median |Humidity9am_median |Humidity3pm_median |Pressure9am_median |Pressure3pm_median |Cloud9am_median |Cloud3pm_median |Temp9am_median |Temp3pm_median |year_median |month_median |day_median |RainTomorrow_indexed_mode|Location_indexed_mode_encoded|WindGustDir_indexed_mode_encoded|WindDir9am_indexed_mode_encoded|WindDir3pm_indexed_mod

<span style="font-size: 36px;color: teal; font-family: TimesNewRoman;">Question 13:	The RainTomorrow is the label, and all other fields are features.<span>

In [135]:
from pyspark.ml.feature import VectorAssembler
# 'encoded_df' is the DataFrame and to assemble all columns except 'RainToday_indexed_mode_encoded' and 'RainTomorrow_indexed_mode_encoded'
feature_columns = [col for col in encoded_df.columns if col not in ['RainTomorrow_indexed_mode']]

assembler = VectorAssembler(inputCols=feature_columns, outputCol="features")
assembled_df = assembler.transform(encoded_df)

assembled_df = assembled_df.drop(*feature_columns)
assembled_df.show()


+-------------------------+--------------------+
|RainTomorrow_indexed_mode|            features|
+-------------------------+--------------------+
|                      0.0|(117,[0,1,2,3,4,5...|
|                      0.0|(117,[0,1,3,4,5,6...|
|                      0.0|(117,[0,1,3,4,5,6...|
|                      0.0|(117,[0,1,3,4,5,6...|
|                      0.0|(117,[0,1,2,3,4,5...|
|                      0.0|(117,[0,1,2,3,4,5...|
|                      0.0|(117,[0,1,3,4,5,6...|
|                      0.0|(117,[0,1,3,4,5,6...|
|                      1.0|(117,[0,1,3,4,5,6...|
|                      0.0|(117,[0,1,2,3,4,5...|
|                      1.0|(117,[0,1,3,4,5,6...|
|                      1.0|(117,[0,1,2,3,4,5...|
|                      1.0|(117,[0,1,2,3,4,5...|
|                      0.0|(117,[0,1,2,3,4,5...|
|                      0.0|(117,[0,1,3,4,5,6...|
|                      1.0|(117,[0,1,3,4,5,6...|
|                      1.0|(117,[0,1,2,3,4,5...|
|                   

<span style="font-size: 36px;color: teal; font-family: TimesNewRoman;">Question 15:	Normalize each numerical column (bring it to a value between 0 and 1)<span>

In [156]:
# Here we gonna normalize the numerical data
from pyspark.ml.feature import StandardScaler
scaled_df = assembled_df
numerical_features = ['features']
for col in numerical_features:
    scaler = StandardScaler(inputCol=col, outputCol=col + "_scaled", withStd=True, withMean=True)
    scaled_df = scaler.fit(scaled_df).transform(scaled_df)
scaled_df = scaled_df.drop(*numerical_features)
scaled_df.show()

+-------------------------+--------------------+
|RainTomorrow_indexed_mode|     features_scaled|
+-------------------------+--------------------+
|                      0.0|[0.19008413095339...|
|                      0.0|[-0.7490394078297...|
|                      0.0|[0.11182383605479...|
|                      0.0|[-0.4673023461948...|
|                      0.0|[0.83181854912190...|
|                      0.0|[0.37790883871003...|
|                      0.0|[0.33095266177087...|
|                      0.0|[-0.7020832308906...|
|                      1.0|[-0.3890420512962...|
|                      0.0|[0.14312795401423...|
|                      1.0|[0.19008413095339...|
|                      1.0|[0.58138560544639...|
|                      1.0|[0.58138560544639...|
|                      0.0|[0.06486765911563...|
|                      0.0|[-0.3733899923165...|
|                      1.0|[0.29964854381143...|
|                      1.0|[0.20573618993311...|
|                   

<span style="font-size: 36px;color: teal; font-family: TimesNewRoman;">Question 16:	Train your Logistic Regression model on the training dataset (70% 30% split).<span>

In [157]:
from pyspark.ml.classification import LogisticRegression
train_df, test_df = scaled_df.randomSplit([0.7,0.3], seed=100)
lr = LogisticRegression(featuresCol='features_scaled', labelCol='RainTomorrow_indexed_mode')
classifier = lr.fit(train_df)

<span style="font-size: 36px;color: teal; font-family: TimesNewRoman;">Question 17:	Predict the RainTomorrow for the test set.<span>

In [167]:
# Evaluate the model on the test data
predictions = classifier.transform(test_df)

<span style="font-size: 36px;color: teal; font-family: TimesNewRoman;">Question 18:	Describe the performance of your model using the confusion matrix and print TP, TN, FP, and FN. In addition, provide the accuracy and F1 score of your model.<span>

In [173]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

# Compute confusion matrix
conf_matrix = predictions.groupBy('RainTomorrow_indexed_mode', 'prediction').count()
conf_matrix.show()

# Extract TP, TN, FP, FN
TP = conf_matrix.filter((conf_matrix.RainTomorrow_indexed_mode == 1) & (conf_matrix.prediction == 1)).select('count').collect()[0]['count']
TN = conf_matrix.filter((conf_matrix.RainTomorrow_indexed_mode == 0) & (conf_matrix.prediction == 0)).select('count').collect()[0]['count']
FP = conf_matrix.filter((conf_matrix.RainTomorrow_indexed_mode == 0) & (conf_matrix.prediction == 1)).select('count').collect()[0]['count']
FN = conf_matrix.filter((conf_matrix.RainTomorrow_indexed_mode == 1) & (conf_matrix.prediction == 0)).select('count').collect()[0]['count']

# Calculate precision
precision = TP / (TP + FP)
print("Precision:", precision)

# Calculate recall
recall = TP / (TP + FN)
print("Recall:", recall)

# Calculate accuracy
accuracy = (TP + TN) / (TP + TN + FP + FN)
print("Accuracy:", accuracy)

# Calculate F1 score using MulticlassClassificationEvaluator
evaluator = MulticlassClassificationEvaluator(labelCol='RainTomorrow_indexed_mode', metricName='f1')
f1_score = evaluator.evaluate(predictions)
print("F1 Score:", f1_score)


+-------------------------+----------+-----+
|RainTomorrow_indexed_mode|prediction|count|
+-------------------------+----------+-----+
|                      1.0|       1.0| 4934|
|                      0.0|       1.0| 1720|
|                      1.0|       0.0| 4717|
|                      0.0|       0.0|31443|
+-------------------------+----------+-----+



Precision: 0.7415088668470093
Recall: 0.5112423583048389
Accuracy: 0.8496519829962161
F1 Score: 0.8390843033229611
